In [22]:
# import of all libraries
from tqdm import tqdm
import itertools
import random
import numpy as np
import threading
import pandas as pd


In [23]:
# create function to read all row in 00-example.txt
def read_file(filename):
    with open(filename) as f:
        return f.read().splitlines()

file1 = read_file('00-example.txt')
print('total rows file 1', len(file1))

file2 = read_file('01-the-cloud-abyss.txt')
print('total rows file 2', len(file2))

file3 = read_file('02-iot-island-of-terror.txt')
print('total rows file 3', len(file3))

file4 = read_file('03-etheryum.txt')
print('total rows file 4', len(file4))

file5 = read_file('04-the-desert-of-autonomous-machines.txt')
print('total rows file 5', len(file5))

file6 = read_file('05-androids-armageddon.txt')
print('total rows file 6', len(file6))

total rows file 1 6
total rows file 2 1001
total rows file 3 3001
total rows file 4 500001
total rows file 5 3001
total rows file 6 100001


In [36]:
file = file4
row1 = file[0].split(' ')
Si = int(row1[0]) # amount of initial stamina
Smax = int(row1[1]) # maximum stamina
T = int(row1[2]) # turns available
D = int(row1[3]) # demons available

print('Si', Si, 'Smax', Smax, 'T', T, 'D', D)

demons = []
for i in range(1, int(D)+1):
    row = file[i].split(' ')
    d = {}
    d['Sc'] = int(row[0]) # stamina cost to face demon
    d['Tr'] = int(row[1]) # Turn required to recover stamina
    d['Sr'] = int(row[2]) # Stamina recovered by defeating demon
    d['Na'] = int(row[3]) # Number of turns you'll earn fragments after defeating demon
    # cast elements of row[4:] to int
    if d['Na'] != 0:
        d['Fa'] = [int(x) for x in row[4:]] # Fragments earned after defeating demon
        # translate d['Fa'] to numpy array
        d['Fa'] = np.array(d['Fa'])
    else:
        d['Fa'] = np.array([])
    demons.append(d)

Si 100 Smax 1000 T 1000 D 500000


In [37]:
# visualize demons in pandas dataframe
df = pd.DataFrame(demons)
# order df from Sr
df = df.sort_values(by=['Sr'], ascending=False)
# reset index
df

,Sc,Tr,Sr,Na,Fa
250392,11,89,40,0,[]
59329,12,55,40,0,[]
347814,11,23,40,0,[]
180210,11,47,40,0,[]
277770,13,19,40,0,[]
...,...,...,...,...,...
43962,7,1,4,0,[]
142880,7,20,4,4,"[120, 400, 400, 200]"
322927,7,120,4,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
472216,7,120,4,10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 540]"


In [43]:
df2 = df.copy()
#substitute Fa array field with the sum of Fa
df2['Fa'] = df2['Fa'].apply(lambda x: x.sum())
# remove Fa == 0 and Sr == 0
df2 = df2[(df2['Fa'] + df2['Sr'] != 0)]
df2

,Sc,Tr,Sr,Na,Fa
250392,11,89,40,0,0.0
59329,12,55,40,0,0.0
347814,11,23,40,0,0.0
180210,11,47,40,0,0.0
277770,13,19,40,0,0.0
...,...,...,...,...,...
43962,7,1,4,0,0.0
142880,7,20,4,4,1120.0
322927,7,120,4,16,2205.0
472216,7,120,4,10,540.0


In [8]:
'''
Create a function to calculate the total fragments earned
'''
def calculate_fragments(seq):
    fragments = 0
    stamina = Si
    demons_defeated = []
    defeated_turns = []
    for i in range(T):
        # create loop for deamons_defeated
        for j in range(len(demons_defeated)):
            # add fragments
            if len(demons[demons_defeated[j]]['Fa']) > i - defeated_turns[j] and len(demons[demons_defeated[j]]['Fa']) > 0:
                fragments += demons[demons_defeated[j]]['Fa'][i - defeated_turns[j]]
            # recharge stamina
            if stamina < Smax and defeated_turns[j] + demons[demons_defeated[j]]['Tr'] == i:
                stamina += demons[demons_defeated[j]]['Sr']
                if stamina > Smax:
                    stamina = Smax
        # defeat demons
        if len(demons_defeated) < len(seq) and stamina >= demons[seq[len(demons_defeated)]]['Sc']:
            stamina -= demons[seq[len(demons_defeated)]]['Sc']
            demons_defeated.append(seq[len(demons_defeated)])
            defeated_turns.append(i)
            if len(demons[demons_defeated[-1]]['Fa']) > 0:
                fragments += demons[demons_defeated[-1]]['Fa'][0]
    return fragments

In [9]:
# generate a random sequence of demons
def generate_random_seq(D):
    seq = random.sample(range(D), D)
    return seq

# Find solution randomly
def find_solution_randomly(D):
    best_seq = []
    best_fragments = 0
    for i in tqdm(range(1000)):
        seq = generate_random_seq(D)
        fragments = calculate_fragments(seq)
        if fragments > best_fragments:
            best_fragments = fragments
            best_seq = seq
    return best_seq, best_fragments
print('Solution randomly')
best_seq, best_fragments = find_solution_randomly(D)
# print('Best sequence', best_seq)
print('Best fragments', best_fragments)

Solution randomly


100%|██████████| 1000/1000 [00:00<00:00, 62237.42it/s]

Best fragments 125


In [17]:
# Calculate the fragments earned by defeating a demon in a specific turn depending on the max turn we have until the end of the game
def fragment_will_obtain(demon, turn_defeated):
    # sum all values in array demons[demon]['Fa'] from 0 to the end of the array or to turn_defeated if array is long enough
    return np.sum(demons[demon]['Fa'][0:min(len(demons[demon]['Fa']), turn_defeated)])
def calculate_sequence_to_optimize_fragment():
    stamina = Si
    demons_defeated = np.array([])
    # create npm array long T
    stamina_recovered = np.array([0]*T)
    for i in tqdm(range(T)):
        # check if stamina is recovered
        stamina += stamina_recovered[i]
        if stamina > Smax:
            stamina = Smax
        # find demon to kill with more fragments
        best_fragments = -1
        best_demon = -1
        for j in range(D):
            # check if demon is not in demons_defeated numpy array
            if j not in demons_defeated:
                # check if stamina is enough to kill demon
                if stamina >= demons[j]['Sc']:
                    # calculate fragments obtained
                    fragments = fragment_will_obtain(j, T-i)
                    # check if fragments obtained are better than the best
                    if fragments > best_fragments:
                        best_fragments = fragments
                        best_demon = j
        if best_demon != -1:
            # kill demon
            stamina -= demons[best_demon]['Sc']
            # add demon to demons_defeated
            demons_defeated = np.append(demons_defeated, best_demon)
            # add stamina recovered
            if demons[best_demon]['Tr'] > 0:
                if len(stamina_recovered) > i+demons[best_demon]['Tr']:
                    stamina_recovered[i+demons[best_demon]['Tr']] += demons[best_demon]['Sr']
            else:
                stamina += demons[best_demon]['Sr']
    return demons_defeated
print('Solution optimized')
best_seq = calculate_sequence_to_optimize_fragment()
print('Best sequence', best_seq)
# cast best_seq to int
best_seq = [int(x) for x in best_seq]
print('Best fragments', calculate_fragments(best_seq))

Solution optimized


100%|██████████| 1000/1000 [00:03<00:00, 280.56it/s]

Best sequence [189. 421.  64. 758. 179. 938. 209. 739. 350. 136. 327. 665. 429. 358.
 210. 404. 508. 185. 495. 289. 812.  96. 607. 811. 787. 606.  20.   8.
  87. 896. 916. 530. 765. 424. 776. 553. 444. 680. 805. 773. 820. 303.
 809. 544. 415. 594. 699. 191. 816. 851. 855.  19. 455. 342. 897. 541.
 447. 309. 290. 982. 567. 702.   0. 250. 143. 716. 885. 988.  59. 305.
 710. 521. 498. 688. 269. 482.  21. 137. 121. 843. 296. 333. 291. 886.
 331. 701. 410. 378. 591. 823. 365. 905. 750. 307. 611.  75.  89. 613.
 641. 134. 672. 501. 322. 123. 859. 540. 737. 159. 719.  36. 200. 946.
 726. 683. 245. 846. 401. 528. 524. 216. 919. 573. 952. 616. 427. 232.
 762. 651. 717. 997. 779. 398.  12. 363. 924. 797. 862. 720.  54. 352.
 237. 201. 756. 793.  67. 909. 713. 218. 246. 880. 985. 760. 584. 546.
 945. 354. 359. 537. 120. 932. 355.  48.  41. 734. 243. 977. 127. 381.
 995.  10. 323. 837. 425. 548. 522. 920. 818. 996. 928.  24. 593. 666.
 543. 686. 437. 638. 163. 391. 335. 568.   6. 377. 282. 397.   

In [98]:
# print sequence in a file txt with 1 row per number of sequence
with open('output.txt', 'w') as f:
    for i in best_seq:
        f.write(str(i) + '\n')

In [73]:
# create a function that get max value of fragments for all possible combinations of demons without repetition
# usable only for first example
def get_max_fragments():
    max_fragments = 0
    max_seq = []
    for seq in tqdm(itertools.permutations(range(D))):
        fragments = calculate_fragments(seq)
        if fragments > max_fragments:
            max_fragments = fragments
            max_seq = seq
    return max_fragments, max_seq
max, seq = get_max_fragments()
print('max fragments', max)
print('sequence', seq)

476it [00:07, 59.57it/s]


KeyboardInterrupt: 

In [ ]:
# print sequence in a file txt with 1 row per number of sequence
with open('output.txt', 'w') as f:
    for i in seq:
        f.write(str(i) + '\n')

In [47]:
t = [1,2,3,4][4]
t

IndexError: list index out of range